In [54]:
import pandas as pd

In [55]:
with open('onto.tsv', 'r') as f:
    #onto = pd.read_csv(f, sep='\t', header=0)
    text = f.read()

# RE


In [56]:
import re
from re import match

### class_to_pred

In [57]:
s = "MemberOf(x,Plant)"
a = re.sub(r'(MemberOf)(\()(\w*)(,)(\w*)(\))', r'\5\2\3\6',s)
a

'Plant(x)'

In [58]:
def class_to_pred(s):
    s = re.sub(r'(MemberOf)(\()(\\|\s|&)*(\w*)(,)(\\|\s|&)*(\w*)(\\|\s|&)*(\))', r'\7\2\4\9',s)
    return s

In [59]:
class_to_pred("MemberOf( \\ &                wind, \\ &                InanimateCarrier \\ &                )")

'InanimateCarrier(wind)'

In [60]:
text = class_to_pred(text)

### Disjoint Union

In [61]:
def partition_to_disjunion(text):
    p = re.compile(r'Partition\((\\|\s|&)*\((\\|\s|&)*((\\*|\s*|&|\w+|,|\s*)*)\.*\)(\\|\s|&)*,(\\|\s|&)*(\w+)(\\|\s|&)*\)')
    regex = re.compile(p)
    for orig_partition in regex.finditer(text):
        orig_partition = orig_partition.group(0)
        print(orig_partition)
        partitions = p.match(orig_partition).group(3).replace(' ','').split(',')
        print(partitions)
        parent = p.match(orig_partition).group(7)
        print(parent)

        a_parent = 'a:'+parent
        a_partitions = str(['\; a:'+part for part in partitions]).replace('[','').replace(']','').replace('\'','').replace('&','').replace('\\','').replace('a: ',' ')
        
        print('HI' , a_parent,a_partitions)

        disj_union = 'DisjointUnion( {a_parent} {a_partitions} )'.format(a_parent=a_parent, a_partitions=a_partitions)
        
        text = text.replace(orig_partition, disj_union)

    return text

In [62]:

s = "Partition(&(\\Flower, \\ Tree, ...),Plant)      \±jfid Partition((Sun, ...),Star)"
partition_to_disjunion(s)

Partition(&(\Flower, \ Tree, ...),Plant)
['Flower', '\\Tree', '']
Plant
HI a:Plant ; a:Flower, ; a:Tree, ; a:
Partition((Sun, ...),Star)
['Sun', '']
Star
HI a:Star ; a:Sun, ; a:


'DisjointUnion( a:Plant ; a:Flower, ; a:Tree, ; a: )      \\±jfid DisjointUnion( a:Star ; a:Sun, ; a: )'

In [63]:
text = partition_to_disjunion(text)

Partition( \\ &                (Organism,\\ &InanimateEntity), \\ &                PathogenCarrier \\ &                )
['Organism', '\\\\&InanimateEntity']
PathogenCarrier
HI a:PathogenCarrier ; a:Organism, ; a:InanimateEntity
Partition( \\ &                (VectingCarrier,\\ &NonVectingCarrier), \\ &                PathogenCarrier \\ &                )
['VectingCarrier', '\\\\&NonVectingCarrier']
PathogenCarrier
HI a:PathogenCarrier ; a:VectingCarrier, ; a:NonVectingCarrier
Partition( \\ &                (Organism,\\ &InanimateEntity), \\ &                PathogenCarrier \\ &                )
['Organism', '\\\\&InanimateEntity']
PathogenCarrier
HI a:PathogenCarrier ; a:Organism, ; a:InanimateEntity
Partition( \\ &                (LivingOrganism,\\ &NonLivingOrganism), \\ &                Organism \\ &                )
['LivingOrganism', '\\\\&NonLivingOrganism']
Organism
HI a:Organism ; a:LivingOrganism, ; a:NonLivingOrganism
Partition( \\ &                (Bacterium,\\ &Fungus,\\ &

## Save File

In [64]:
with open('onto_out.tsv', 'w+') as f:
    #onto = pd.read_csv(f, sep='\t', header=0)
    f.write(text)